In [1]:
!pip install tensorflow==2.15

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.15.0'

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [6]:
!kaggle datasets download -d vijaygiitk/multiclass-weather-dataset

Dataset URL: https://www.kaggle.com/datasets/vijaygiitk/multiclass-weather-dataset
License(s): CC0-1.0
100% 134M/134M [00:01<00:00, 124MB/s]
100% 134M/134M [00:01<00:00, 116MB/s]


In [7]:
!unzip '/content/multiclass-weather-dataset.zip'

Archive:  /content/multiclass-weather-dataset.zip
  inflating: dataset/alien_test/Cloud_1.png  
  inflating: dataset/alien_test/Cloud_2.jpg  
  inflating: dataset/alien_test/Cloud_3.jpeg  
  inflating: dataset/alien_test/Cloud_4.jpg  
  inflating: dataset/alien_test/foggy_1.jpg  
  inflating: dataset/alien_test/foggy_10.jpg  
  inflating: dataset/alien_test/foggy_2.jpg  
  inflating: dataset/alien_test/foggy_3.jpg  
  inflating: dataset/alien_test/foggy_4.jpg  
  inflating: dataset/alien_test/foggy_5.jpg  
  inflating: dataset/alien_test/foggy_6.jpg  
  inflating: dataset/alien_test/foggy_7.jpg  
  inflating: dataset/alien_test/foggy_8.jpg  
  inflating: dataset/alien_test/foggy_9.jpg  
  inflating: dataset/alien_test/rain_1.jpg  
  inflating: dataset/alien_test/rain_2.png  
  inflating: dataset/alien_test/rain_3.jpg  
  inflating: dataset/alien_test/rain_4.jpg  
  inflating: dataset/alien_test/rain_5.jpg  
  inflating: dataset/alien_test/rain_6.jpg  
  inflating: dataset/alien_test/sh

In [8]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import ImageFile

In [9]:
import os
import shutil
import random

def split_dataset(dataset_dir, test_ratio=0.2):
    categories = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']
    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        train_path = os.path.join(dataset_dir, 'train', category)
        test_path = os.path.join(dataset_dir, 'test', category)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)
        files = os.listdir(category_path)
        random.shuffle(files)
        split_index = int(len(files) * (1 - test_ratio))
        train_files = files[:split_index]
        test_files = files[split_index:]
        for file in train_files:
            shutil.move(os.path.join(category_path, file), os.path.join(train_path, file))
        for file in test_files:
            shutil.move(os.path.join(category_path, file), os.path.join(test_path, file))

dataset_dir = '/content/dataset'
split_dataset(dataset_dir)


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=[0.99, 1.01],
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    data_format="channels_last",
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
training_set = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    '/content/dataset/test',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)

Found 1200 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [12]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(180, 180, 3))


80134624/80134624 [==============================] - 0s 0us/step


In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [16]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [17]:
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/30
19/19 [==============================] - 37s 1s/step - loss: 4.0534 - accuracy: 0.4475 - val_loss: 0.7768 - val_accuracy: 0.6867
Epoch 2/30
19/19 [==============================] - 16s 844ms/step - loss: 0.5519 - accuracy: 0.8000 - val_loss: 0.5901 - val_accuracy: 0.8067
Epoch 3/30
19/19 [==============================] - 16s 831ms/step - loss: 0.3770 - accuracy: 0.8683 - val_loss: 0.4675 - val_accuracy: 0.8533
Epoch 4/30
19/19 [==============================] - 16s 836ms/step - loss: 0.2711 - accuracy: 0.9158 - val_loss: 0.4457 - val_accuracy: 0.8700
Epoch 5/30
19/19 [==============================] - 16s 838ms/step - loss: 0.2068 - accuracy: 0.9383 - val_loss: 0.3503 - val_accuracy: 0.8667
Epoch 6/30
19/19 [==============================] - 16s 833ms/step - loss: 0.1623 - accuracy: 0.9583 - val_loss: 0.3563 - val_accuracy: 0.8933
Epoch 7/30
19/19 [==============================] - 16s 843ms/step - loss: 0.1464 - accuracy: 0.9583 - val_loss: 0.3686 - val_accuracy: 0.8767
Ep

In [18]:

loss, accuracy = model.evaluate(
    test_set,
    steps=len(test_set),
    verbose=2,
    use_multiprocessing=True,
    workers=2
)

print(f'Model performance on test images: \nAccuracy = {accuracy}\nLoss = {loss}')

model.save('vgg19_model.h5')


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


5/5 - 2s - loss: 0.4631 - accuracy: 0.8667 - 2s/epoch - 386ms/step
Model performance on test images: 
Accuracy = 0.8666666746139526
Loss = 0.46312215924263


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('/content/vgg19_model.h5')

img_path = '/content/download (1).jpeg'
img = image.load_img(img_path, target_size=(180, 180))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
pred = np.argmax(preds, axis=1)

index = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']

result = index[pred[0]]
print(f'The model predicts the image is: {result}')


1/1 [==============================] - 0s 154ms/step
The model predicts the image is: shine
